### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import snowflake.connector as sfc

### Postgres setup

In [2]:
pg_schema = 'com_zx_courier'
pg_table = 'stg_order'

pg_date_col = 'created_date'
pg_id_col = 'id'
pg_order_no_col = 'order_number'


### Snowflake setup

In [3]:
sf_schema = 'BR_ZE'
sf_table = 'STR_ORDER'

sf_date_col = 'created_date'
sf_id_col = 'id'
sf_order_no_col = 'order_number'

### Postgres connection and query

In [4]:
def gdw_query(query):
    conn = psycopg2.connect(
                    user='postgres',
                    password='',
                    host='localhost',
                    port='5432',
                    database='gdwprod')
    try:
        df = pd.read_sql_query(query, conn)
        return df
    finally:
        conn.close()

In [30]:
pg_query1 = '''select min({date_col}) first_entry,\
                    max({date_col}) last_entry,\
                    min({id_col}) first_id,\
                    max({id_col}) last_id,\
                    min({order_no_col}) first_order_number,\
                    max({order_no_col}) last_order_number,\
                    count(distinct {id_col}) num_id,\
                    count(distinct {order_no_col}) num_order_numbers,\
                    count(*) num_entries
                from {schema}.{table};'''.format(
                                                schema=pg_schema, 
                                                table=pg_table,
                                                date_col=pg_date_col,
                                                id_col=pg_id_col,
                                                order_no_col=pg_order_no_col)

pg_query2 = '''select min({date_col}) first_entry,\
                    max({date_col}) last_entry,\
                    min({id_col}) first_id,\
                    max({id_col}) last_id,\
                    min({order_no_col}) first_order_number,\
                    max({order_no_col}) last_order_number,\
                    count(distinct {id_col}) num_id,\
                    count(distinct {order_no_col}) num_order_numbers,\
                    count(*) num_entries\
            from {schema}.{table}\
            where date_trunc('month', date({date_col})) < date_trunc('month', date('2020-03-01'));'''.format(
                                                                                            schema=pg_schema, 
                                                                                            table=pg_table,
                                                                                            date_col=pg_date_col,
                                                                                            id_col=pg_id_col,
                                                                                            order_no_col=pg_order_no_col)


pg_query3 = '''select date_trunc('month', date({date_col})) which_month,\
                    count(distinct {id_col}) num_id,\
                    count(*) num_entries\
                from {schema}.{table} group by date_trunc('month', date({date_col})) 
                order by date_trunc('month', date({date_col}));'''.format(
                                                                    schema=pg_schema, 
                                                                    table=pg_table,
                                                                    date_col=pg_date_col,
                                                                    id_col=pg_id_col,
                                                                    order_no_col=pg_order_no_col)

### Snowflake connection and query

In [27]:
def snow_query(query):
    conn = sfc.connect(
                    user='sbhardwaj',
                    password='Satainder@334',
                    account='zxventures.us-east-1',
                    warehouse='WH_INTERACTIVE',
                    database='OMNICHANNEL',)
    try:
        df = pd.read_sql_query(query, conn)
        return df
    finally:
        conn.close()

In [36]:
sf_query1 = '''select min({date_col}) first_entry,\
                    max({date_col}) last_entry,\
                    min({id_col}) first_id,\
                    max({id_col}) last_id,\
                    min({order_no_col}) first_order_number,\
                    max({order_no_col}) last_order_number,\
                    count(distinct {id_col}) num_id,\
                    count(distinct {order_no_col}) num_order_numbers,\
                    count(*) num_entries
                from {schema}.{table};'''.format(
                                                schema=sf_schema, 
                                                table=sf_table,
                                                date_col=sf_date_col,
                                                id_col=sf_id_col,
                                                order_no_col=sf_order_no_col)

sf_query2 = '''select min({date_col}) first_entry,\
                    max({date_col}) last_entry,\
                    min({id_col}) first_id,\
                    max({id_col}) last_id,\
                    min({order_no_col}) first_order_number,\
                    max({order_no_col}) last_order_number,\
                    count(distinct {id_col}) num_id,\
                    count(distinct {order_no_col}) num_order_numbers,\
                    count(*) num_entries\
            from {schema}.{table}\
            where date_trunc('month', date({date_col})) < date_trunc('month', date('2020-03-01'));'''.format(
                                                                                            schema=sf_schema, 
                                                                                            table=sf_table,
                                                                                            date_col=sf_date_col,
                                                                                            id_col=sf_id_col,
                                                                                            order_no_col=sf_order_no_col)


sf_query3 = '''select date_trunc('month', date({date_col})) which_month,\
                    count(distinct {id_col}) num_id,\
                    count(*) num_entries\
                from {schema}.{table} group by date_trunc('month', date({date_col}))
                order by date_trunc('month', date({date_col}));'''.format(
                                                                    schema=sf_schema, 
                                                                    table=sf_table,
                                                                    date_col=sf_date_col,
                                                                    id_col=sf_id_col,
                                                                    order_no_col=sf_order_no_col)

## Various overall metrics

In [19]:
pg1 = gdw_query(pg_query1)
sf1 = snow_query(sf_query1)
pg1

,first_entry,last_entry,first_id,last_id,first_order_number,last_order_number,num_id,num_order_numbers,num_entries
0,2015-12-05 14:27:44,2020-06-12 04:45:18.812269,18,8617296,00000004,99999993,8461469,8461468,8461470


In [20]:
sf1

,FIRST_ENTRY,LAST_ENTRY,FIRST_ID,LAST_ID,FIRST_ORDER_NUMBER,LAST_ORDER_NUMBER,NUM_ID,NUM_ORDER_NUMBERS,NUM_ENTRIES
0,2015-12-05 14:27:44,2020-06-12 00:06:15.821029,18,8596288,1,99999985,7981765,7977189,7981765


In [21]:
str_order_metrics = pd.DataFrame(columns=['Postgres', 'Snowflake', 'is_identical'], index=pg1.columns)
str_order_metrics['Postgres'] = pg1.T.values
str_order_metrics['Snowflake'] = sf1.T.values
str_order_metrics['is_identical'] = str_order_metrics['Postgres'] == str_order_metrics['Snowflake']

str_order_metrics

,Postgres,Snowflake,is_identical
first_entry,2015-12-05 14:27:44,2015-12-05 14:27:44,True
last_entry,2020-06-12 04:45:18.812269,2020-06-12 00:06:15.821029,False
first_id,18,18,True
last_id,8617296,8596288,False
first_order_number,00000004,1,False
last_order_number,99999993,99999985,False
num_id,8461469,7981765,False
num_order_numbers,8461468,7977189,False
num_entries,8461470,7981765,False


## Various overall metrics - before March 2020

In [22]:
pg2 = gdw_query(pg_query2)
sf2 = snow_query(sf_query2)
pg2

,first_entry,last_entry,first_id,last_id,first_order_number,last_order_number,num_id,num_order_numbers,num_entries
0,2015-12-05 14:27:44,2020-02-29 23:59:59.954054,18,3345422,00000119,99999985,3193382,3193381,3193382


In [23]:
sf2

,FIRST_ENTRY,LAST_ENTRY,FIRST_ID,LAST_ID,FIRST_ORDER_NUMBER,LAST_ORDER_NUMBER,NUM_ID,NUM_ORDER_NUMBERS,NUM_ENTRIES
0,2015-12-05 14:27:44,2020-02-29 23:59:59.954054,18,3345422,1,99999985,3193382,3191575,3193382


In [24]:
str_order_metrics = pd.DataFrame(columns=['Postgres', 'Snowflake', 'is_identical'], index=pg1.columns)
str_order_metrics['Postgres'] = pg2.T.values
str_order_metrics['Snowflake'] = sf2.T.values
str_order_metrics['is_identical'] = str_order_metrics['Postgres'] == str_order_metrics['Snowflake']

str_order_metrics

,Postgres,Snowflake,is_identical
first_entry,2015-12-05 14:27:44,2015-12-05 14:27:44,True
last_entry,2020-02-29 23:59:59.954054,2020-02-29 23:59:59.954054,True
first_id,18,18,True
last_id,3345422,3345422,True
first_order_number,00000119,1,False
last_order_number,99999985,99999985,False
num_id,3193382,3193382,True
num_order_numbers,3193381,3191575,False
num_entries,3193382,3193382,True


## Month-wise number of orders

In [33]:
pg3 = gdw_query(pg_query3)
pg3.index = pd.to_datetime(pg3.which_month)
pg3.head()

,which_month,num_id,num_entries
which_month,,,
2015-12-01 00:00:00+00:00,2015-12-01 00:00:00+00:00,72,72
2016-01-01 00:00:00+00:00,2016-01-01 00:00:00+00:00,92,92
2016-02-01 00:00:00+00:00,2016-02-01 00:00:00+00:00,138,138
2016-03-01 00:00:00+00:00,2016-03-01 00:00:00+00:00,211,211
2016-04-01 00:00:00+00:00,2016-04-01 00:00:00+00:00,335,335


In [37]:
sf3 = snow_query(sf_query3)
sf3.index = pd.to_datetime(sf3['WHICH_MONTH'])
sf3.head()

,WHICH_MONTH,NUM_ID,NUM_ENTRIES
WHICH_MONTH,,,
2015-12-01,2015-12-01,72,72
2016-01-01,2016-01-01,92,92
2016-02-01,2016-02-01,138,138
2016-03-01,2016-03-01,211,211
2016-04-01,2016-04-01,335,335


In [47]:
str_order_monthwise= pd.DataFrame(columns=['Postgres', 'Snowflake', 'is_identical'], index=pg3.index)
str_order_monthwise.index.name = 'Month'
str_order_monthwise['Postgres'] = pg3['num_entries'].values
str_order_monthwise['Snowflake'] = sf3['NUM_ENTRIES'].values
str_order_monthwise['is_identical'] = str_order_monthwise.Postgres == str_order_monthwise.Snowflake

str_order_monthwise.head()

,Postgres,Snowflake,is_identical
Month,,,
2015-12-01 00:00:00+00:00,72,72,True
2016-01-01 00:00:00+00:00,92,92,True
2016-02-01 00:00:00+00:00,138,138,True
2016-03-01 00:00:00+00:00,211,211,True
2016-04-01 00:00:00+00:00,335,335,True


In [48]:
str_order_monthwise[str_order_monthwise['is_identical']!= True]

,Postgres,Snowflake,is_identical
Month,,,
2020-03-01 00:00:00+00:00,705903,247186,False
2020-06-01 00:00:00+00:00,830969,809981,False
